In [1]:
import cv2
import h5py
from pathlib import Path
import random
import os
import sys

# F-RCNN Dataset Generation

## Importing the function

In [2]:
sys.path.append("..")  # TODO: Fix it!
from dataset.create_dataset import create_dataset

2024-07-25 13:05:16.368507: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 13:05:16.391247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 13:05:16.416870: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 13:05:16.424594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 13:05:16.442719: I tensorflow/core/platform/cpu_feature_guar

## Defining the path

A `.h5` file containing a F-RCNN example is provided in this example (`FRCNN_dataset.h5`).

The following F-RCNN dataset was created using the image annotation tool available [here](https://github.com/sinmec/multilabellerg).

The current implementation can read multiple `.h5` files. This is done by simply placing the dataset files in the same `h5_dataset_path` folder.

The images are extracted to the `output_path` folder.

In [3]:
h5_dataset_path = Path('/home/rafaelfc/Data/PIV_UNET_IJMF_DATASET/')
output_path = Path(r"dataset_FRCNN_PIV_IJMF_2")

## Defining parameters

In [4]:
# Number of Validation images
N_VALIDATION = 5

# Number of Verification images
N_VERIFICATION = 5

For the dataset generation, two options are provided.

In the first one, the dataset is composed of the annotated contours and the original images, i.e., the F-RCNN object detection is based on the original image information. This option may work well if the image is not complex and the objects are easily distinguished.
For this option, simply define the variable  `UNET_model_options = None`, as shown (and commented) below.

In [1]:
UNET_model_options = None

In some cases, for instance, when dealing with [bubbly flow PIV acqusitions](https://www.sciencedirect.com/science/article/pii/S0009250918303269), the objects are not readily visible in the original image.
In this case, the F-RCNN object detection is based on an intermediate representation of the original image. Here, we apply a U-Net model to segment the image and highlight the bubble positions. For this option, the `UNET_model_options` should be configured.
It is important to note that, to use this option, it is required to train an appropriate U-Net model, following the guidelines detailed in this repository.

In [5]:
UNET_model_options = {'keras_file': '/home/rafaelfc/Data/DeepFlowImaging/UNET/training/keras-tuner/UNET_best_model_large_01.keras',
                      'window_size': 512,
                      'sub_image_size': 128,
                      'stride_division': 16}                      

## Generating the dataset

The dataset is generated from the `create_dataset` function. Additional options include:  

 - `limit_height` This is useful for long images, such as those acquired from straight pipe sections. Example `limit_height=600`

In [6]:
create_dataset(h5_dataset_path, output_path, N_VALIDATION, N_VERIFICATION, UNET_model_options=UNET_model_options, limit_height=600)

2024-07-25 13:05:19.344809: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3859 MB memory:  -> device: 0, name: NVIDIA RTX A2000, pci bus id: 0000:b3:00.0, compute capability: 8.6
Creating dataset:   0%|                                                                                                                                             | 0/1 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1721923520.238197  955736 service.cc:146] XLA service 0x7f67940031b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721923520.238252  955736 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A2000, Compute Capability 8.6
2024-07-25 13:05:20.270297: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to en

By running this function, you'll see that it created three new folders on the chosen path:
 - Training: Samples used during the F-RCNN training
 - Validation: Samples used for validation purpouses during training - Total of `N_VALIDATION` full images
 - Verification: Samples used to evaluate the F-RCNN accuracy after the training step. Unseen data during training  - Total of `N_VERIFICATION` full images



## Option 1
**If the `UNET_model_options` <span style="color:red">is not</span> defined**, the F-RCNN object detection is based on the original image. 
In this case, folder cotains 3 folders:
 - `contours`: Text files which contains coordinates and parameters of the contours
 - `debug`: All the labelled images from the `.h5` file
 - `images`: All the raw images from the `.h5` file

## Option 2
**If the `UNET_model_options` <span style="color:red">is </span> defined**, the F-RCNN object detection is based on the U-Net (segmented) representation of the image. 
In this case, folder cotains 4 folders:
 - `contours`: Text files which contains coordinates and parameters of the contours
 - `debug`: All the labelled images from the `.h5` file
 - `images`: All the raw images from the `.h5` file
 - `masks`: U-Net segmented images from images originally labeled in the `.h5` file

In both options, the folder created in this step should be sent to the `FRCNN/dataset` folder for training.